# correcting_lat_long

Notebook to correct erroneous longitude and latitude values found on the corrected dataset

In [155]:
# Libraries

import requests
import pandas as pd
import pickle
import json
import numpy as np



import sys
sys.path.insert(0, '..')

In [159]:
path = '../data/Boston_Housing_Market_Challenge/boston_corrected.csv'

df = pd.read_csv(path)


df.head()

,TOWN,TOWNNO,TRACT,LON,LAT,MEDV,CMEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,Nahant,0,2011,-70.955,42.2550,24.0,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,Swampscott,1,2021,-70.950,42.2875,21.6,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,Swampscott,1,2022,-70.936,42.2830,34.7,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,Marblehead,2,2031,-70.928,42.2930,33.4,33.4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,Marblehead,2,2032,-70.922,42.2980,36.2,36.2,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [160]:
API_key = json.load(open('../config.json', 'rb'))['Google_API_key']

root_path = 'https://maps.googleapis.com/maps/api/geocode/json?address=+{},+USA&key={}'

get_response = lambda name : requests.get(root_path.format(name, key)).json()
lat_long = []

town_names = np.unique(df.TOWN)
for town in town_names:
    response = requests.get(
        root_path.format(town, API_key)
    )
    lat_long.append(list(response.json()['results'][0]['geometry']['location'].values()))
    
    if town == 'Nahant':
        # verify Nahant
        print(list(response.json()['results'][0]['geometry']['location'].values()))

# verify correct number
len(lat_long)

[42.4265633, -70.922279]


92

In [161]:
lat_long[:5]

[[38.8799697, -77.1067698],
 [42.1945758, -122.7094767],
 [42.4906231, -71.2760089],
 [37.5202145, -122.2758008],
 [42.5584283, -70.880049]]

In [162]:
pickle.dump(lat_long, open('../data/other_data/lat_long_centers.pkl', 'wb'))

In [163]:
lat_long_values = []

wrong_values = df.groupby('TOWN').mean() # based on the mean

for name, (lat, long) in zip(town_names, lat_long):
    df_temp = df[df.TOWN == name]
    mean_temp = wrong_values[wrong_values.index == name][['LAT', 'LON']].values[0]
    for lat_local, long_local in df_temp[['LAT', 'LON']].values:
        diff = [lat_local - mean_temp[0], long_local - mean_temp[1]]
        lat_long_values.append([lat + diff[0], long + diff[1]]) 

In [164]:
lat_long_values = np.asarray(lat_long_values)
df = df.sort_values('TOWN')
df[['LAT', 'LON']] = lat_long_values
df.head()

,TOWN,TOWNNO,TRACT,LON,LAT,MEDV,CMEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
178,Arlington,29,3567,-77.098798,38.872313,29.9,29.9,0.06642,0.0,4.05,0,0.51,6.860,74.4,2.9153,5,296,16.6,391.27,6.92
177,Arlington,29,3566,-77.097298,38.875713,24.6,24.6,0.05425,0.0,4.05,0,0.51,6.315,73.4,3.3175,5,296,16.6,395.60,6.29
176,Arlington,29,3565,-77.095098,38.878213,23.2,23.2,0.07022,0.0,4.05,0,0.51,6.020,47.2,3.5549,5,296,16.6,393.23,10.11
175,Arlington,29,3564,-77.105798,38.888213,29.4,29.4,0.06664,0.0,4.05,0,0.51,6.546,33.1,3.1323,5,296,16.6,390.96,5.33
174,Arlington,29,3563,-77.124298,38.885713,22.6,22.6,0.08447,0.0,4.05,0,0.51,5.859,68.7,2.7019,5,296,16.6,393.23,9.64


In [165]:
# check Nahant
df[df.TOWN == 'Nahant']

,TOWN,TOWNNO,TRACT,LON,LAT,MEDV,CMEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,Nahant,0,2011,-70.922279,42.426563,24.0,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98


In [107]:
df.to_csv('boston_fixed.csv')